Visualizing imaginary quadratic number fields

Packages to import

In [4]:
import json
import requests
import numpy as np
from cmath import sqrt
import math
import matplotlib.pyplot as plt

Helper functions

In [5]:
# Helper function to obtain discriminants within a certain range based on data provided by the lmfdb
def get_disc(lower, upper):
    disc_data=[]
    disc_list=range(lower,upper+1) #list of possible discriminants
    for disc in disc_list:
        URL_TRUNK = ("https://www.lmfdb.org/api/nf_fields/?_format=json&degree=2&r2=1&disc_sign=-1&disc_abs="+str(disc)+
             "&_fields=label,disc_abs,disc_sign")
        link = URL_TRUNK.format(str(2)) 
        dataset = requests.get(url=link).json()["data"]
        if dataset != []:
            disc_data.append(dataset[0]['disc_abs']*dataset[0]['disc_sign'])
    return(disc_data)

# Helper function to obtain d square of generator from given discriminant
def obtain_d(dis):
    if dis % 4 == 1:
        d = dis
    else: d = dis/4
    return d      

# Helper function to compute integral basis from square of generator.
# Based on AZT script and Stange 2. Notations
def tau(d):
    if d%4==1:
        return (1+sqrt(d))/2
    return sqrt(d)

# Helper function to compute norm of a complex number
def norm(a):
    return (a*a.conjugate()).real

# Helper function. 
# Determines for a given complex number x and a given d if x in O_d
def in_O(x,d):
    if d%4==1:
        b=(x.imag/(sqrt(-d)/2)).real 
        if(int(b)==b and int(x.real-(b/2))==x.real-(b/2)):
            return True
    if d%4!=1:
        b=(x.imag/(sqrt(-d))).real 
        if(int(b)==b and int(x.real)==x.real):
            return True
    return False

# Define lattice between lower and upper bound
def bound_list(lower_bound,upper_bound,Tau):
    r=[a+b*Tau for a in range(-upper_bound,upper_bound+1) for b in range(-upper_bound, upper_bound+1) if lower_bound<=norm(a+b*Tau)<=upper_bound]
    return r

Functions for computing and plotting circles

In [6]:
# Computes matrices - based on what?
def compute_matrices(disc, D, Tau, curv):
    matrices = []
    #beta=[a+b*Tau for a in range(-curv,curv) for b in range(-curv) if norm(a+b*Tau)<=(curv**2)]
    beta=[a+b*Tau for a in range(math.floor(-curv/2),math.ceil(curv/2+1)) for b in range(math.floor(-curv/(2*sqrt(-D).real)),math.ceil(curv/(2*sqrt(-D).real)+1)) if norm(a+b*Tau)<=(curv**2)]
    #print(len(beta))
    for b in beta:
        #print(beta.index(b))
        if(norm(b)!=0):
            #alpha=bound_list(0,math.ceil((4+abs(disc))*norm(b)),Tau)
            alpha=bound_list(0,math.ceil((4+abs(disc))*norm(b)),Tau)
            #delta=bound_list(norm(b),math.ceil(4*curv**2),Tau)
            delta=bound_list(0,math.ceil(4*curv**2/(3*norm(b))),Tau)
            for d in delta:
                #if((d/b).real>1/2):
                 #  continue
                # Ueberpruefe: liegt die Kruemmung in den Grenzen der vorgegebenen?
                if (-curv<=(1j*(b*d.conjugate()-b.conjugate()*d)).real <=curv):
                    for a in alpha:
                        if in_O((a*d-1)/b,D):
                            matrices.append((a,b,(a*d-1)/b,d))
    return matrices


# Alternative function for computing circles
# Mirrors circles on axes and origin
# Based on Stange 3.7
def compute_circles(matrices):
    temp = []
    circles = []
    for m in matrices:
        a,b,c,d=m
        curv=1j*(b*d.conjugate()-b.conjugate()*d)
        if(curv!=0):
            # Berechnet Mittelpunkt des Kreises
            z=1j*(a*d.conjugate()-c*b.conjugate())/(curv)
            x=z.real
            y=z.imag
            # Berechnet Radius
            r=(1/curv).real
            # Adds center and radius
            temp.append(((round(x,3),round(y,3)),round(r,3)))
            # Mirror circles on axes and origin
    for data in temp:
        if data not in circles:
            circles.append(data)
        if data[0][0] != 0 and ((-data[0][0],data[0][1]),data[1]) not in circles:
            circles.append(((-data[0][0],data[0][1]),data[1]))
        if data[0][1] != 0 and ((data[0][0],-data[0][1]),data[1]) not in circles:
            circles.append(((data[0][0],-data[0][1]),data[1]))
        if data[0][0] != 0 and data[0][1] != 0 and ((-data[0][0],-data[0][1]),data[1]) not in circles:
            circles.append(((-data[0][0],-data[0][1]),data[1]))
    # remove recurring circles
    circles=list(set(circles))
    
    return circles
    
    
# Filter circles for center of the circle
# Woher die Grenzen?
def filter_circles(circles, D):
    circles_filter=[]
    for circle in circles:
        # Filtern nach Ort des Mittelpunktes und Groesse des Radius
        if (-1<=circle[0][0]<=1 and -1<=circle[0][1]<=1):#sqrt(-D).real):
            circles_filter.append(circle)
    return circles_filter


# Filter circles for fundamental parallelogram
def filter_circles_quadrangular(circles, D, Tau):
    circles_filter = []
    for circle in circles:
        b = circle[0][1]/Tau.imag
        if 0 <= b <= 1:
            if 0 <= circle[0][0] - b*Tau.real <= 1:
                circles_filter.append(circle)
    return circles_filter


# Filter circles circularly 
def filter_circles_circular(circles, D, xctr=0, yctr=0.5, radius=0.25):
    circles_filter = []
    for circle in circles:
        if circle[1] > 0 and (circle[0][0] - xctr)**2 + (circle[0][1] - yctr)**2 < radius:
            circles_filter.append(((circle[0][0], circle[0][1]), circle[1]))
    return circles_filter


# plot circles
def plot_circles_quadrangular(circles_filter, D):
    fig=plt.figure(figsize=(10,10))
    ax=plt.subplot(aspect='equal')

    # plot circles
    for circle in circles_filter:
        circle1 = plt.Circle((circle[0][0], circle[0][1]), circle[1], color='b', fill=False)
        
        plt.gca().add_patch(circle1)
    
    plt.xlim(left = -1.5, right = 1.5)
    plt.ylim(bottom = -sqrt(-D).real, top = sqrt(-D).real)
    plt.show()
    
    
    
# plot mandala
def plot_circles_circular(circles_filter, D):
    fig=plt.figure(figsize=(10,10))
    ax=plt.subplot(aspect='equal')

    # plot circles
    for circle in circles_filter:
        circle1 = plt.Circle((circle[0][0], circle[0][1]), circle[1], color='b', fill=False)
        
        plt.gca().add_patch(circle1)
    
    plt.xlim(left = -1, right = 1)
    plt.ylim(bottom = -1, top = 2)
    plt.axis('off')
    plt.show()
    
    
def final_picture_mandala(disc, curv, xctr, yctr, radius):
    D = obtain_d(disc) # square root of generator
    Tau = tau(D) # integral basis
    
    matrices = compute_matrices(disc, D, Tau, curv)
    circles = compute_circles(matrices)
    circles_filter = filter_circles_circular(circles, D, xctr, yctr, radius)
    plot_circles_circular(circles_filter, D)
    
    
# Plotting circles within fundamental parallelogram
def final_picture_parallelogram(disc, curv):
    D = obtain_d(disc) # square root of generator
    Tau = tau(D) # integral basis
    
    matrices = compute_matrices(disc, D, Tau, curv)
    circles = compute_circles(matrices)
    circles_filter = filter_circles_quadrangular(circles, D, Tau)
    #print(len(circles_filter))
    plot_circles_quadrangular(circles, D)
    
    
    
# Wrapping everything up in a single function
def visualize_mandala(upper, lower=0, curv=15, xctr=0, yctr=0.5, radius=0.25):
    disc_data = get_disc(lower, upper)
    for disc in disc_data:
        print("Working on discriminant", disc, ". ", len(disc_data)-disc_data.index(disc), " more discriminants to go.")
        final_picture_mandala(disc, curv, xctr, yctr, radius)
    
    
def visualize_parallelogram(upper, lower=0, curv=15):
    disc_data = get_disc(lower, upper)
    for disc in disc_data:
        print("Working on discriminant", disc, ". ", len(disc_data)-disc_data.index(disc), " more discriminants to go.")
        final_picture_mandala(disc, curv)